In [1]:
%run __init__.ipynb

Successfully connected to MongoDB


In [2]:
#Highlighted KPIS
def domain_108(df):
    domain = df.copy()
    #Calcul US
    domain = domain.groupby('Date', as_index=False).agg({'Confirmed':'sum', 'Deaths':'sum' , 'Population':'sum'})
    domain.loc[(domain['Deaths'] != 0),'Fatality Rate'] = round((domain['Deaths']/domain['Confirmed']*100),2)
    domain.loc[(domain['Confirmed'] != 0),'Confirmed Ratio'] = round((domain['Confirmed']/domain['Population']*100000),0)
    domain.loc[(domain['Deaths'] != 0),'Deaths Ratio'] = round((domain['Deaths']/domain['Population']*100000),0)
    
    #Create DF for HKPIS
    cols_to_keep = ['Date']
    domain= domain.melt(id_vars=cols_to_keep,value_name='VALUE',var_name='KPI')
    
    #Calc variation
    kpis = domain['KPI'].drop_duplicates().values.tolist()
    final_domain = pd.DataFrame()
    for kpi in kpis:
        tmp_df = domain[domain['KPI'] == kpi].sort_values(by="Date", ascending=False).reset_index(drop=True)
        final_domain = final_domain.append(tmp_df)
    
    #Create df last + merge
    final_domain['D'] = (final_domain.index).astype(str)
    final_domain['D-1'] = (final_domain.index + 1).astype(str)
    cols_to_rename = {"D": "D-1",'VALUE':'VALUE_D-1'}
    df_last = final_domain[['D','KPI','VALUE']].rename(index=str, columns=cols_to_rename) 
    final_domain = final_domain.merge(df_last, on= ['D-1','KPI'], how= 'left').drop(['D','D-1'], axis=1)
    
    #Add fields 
    #SCENARIO
    final_domain['SCENARIO'] = pd.to_datetime(final_domain['Date'], format='%m/%d/%y').dt.strftime('%d/%m/%Y')
    
    #VARIATION
    final_domain['VAR'] = final_domain['VALUE'] - final_domain['VALUE_D-1']
    final_domain['VARP'] = final_domain['VAR'] / abs(final_domain['VALUE_D-1']) * 100
    
    #UNIT & PRECISION
    final_domain.loc[:,'UNIT_VALUE'] = ''
    final_domain.loc[final_domain['KPI'] == 'Fatality Rate','UNIT_VALUE'] = ' %'
    final_domain.loc[:,'UNIT_VAR'] = ''
    final_domain.loc[final_domain['KPI'] == 'Fatality Rate','UNIT_VAR'] = ' pts'
    final_domain.loc[:,'PRECISION_VALUE'] = ',.0f'
    final_domain.loc[final_domain['KPI'] == 'Fatality Rate','PRECISION_VALUE'] = ',.2f'
    final_domain.loc[:,'PRECISION_VAR'] = '+,.0f'
    final_domain.loc[final_domain['KPI'] == 'Fatality Rate','PRECISION_VAR'] = '+,.3f'
    
    final_domain = final_domain.drop('Date',axis=1).dropna().reset_index(drop=True)
    if not final_domain.empty:
        naas_drivers.mongo.send(final_domain, '108', DB_APP, True)
    return final_domain

In [7]:
%%time

# Load Data Source
def check_data():
    # Data check
    file_path_check = os.path.join(OUTPUT_FOLDER,'US_MAP.csv')
    df_check = pd.DataFrame()
    if path.exists(file_path_check):
        df_check = pd.read_csv(file_path_check, sep=";")
        df_check = df_check[(df_check['Date'] == YESTERDAY.strftime('%d/%m/%y'))].reset_index(drop=True)
        if df_check.empty:
            print(f'Data already updated !')
            %stop 
    return df_check


# Trasnform Data Source
def transform_data(df):
    #--Create Dataframe
    def create_df(df_name, kpi, agg, cols_to_keep):
        file_path_conf = os.path.join(OUTPUT_FOLDER, f'{df_name}.csv')
        df = pd.read_csv(file_path_conf, sep=";")
        df = df.drop('Unnamed: 0',axis=1)
        df = df.melt(id_vars=cols_to_keep,value_name=kpi,var_name='Date')
        df = df.groupby(['Date','Province_State'], as_index=False).agg(agg)
        return df
    
    cols_to_keep = ['UID','iso2', 'iso3',
                    'code3','FIPS','Admin2',
                    'Lat','Long_','Combined_Key',
                    'Province_State','Country_Region']
    c_us = create_df('US_CONFIRMED', 'Confirmed' , {'Confirmed':'sum'}, cols_to_keep)
    
    cols_to_keep = ['UID','iso2', 'iso3',
                    'code3','FIPS','Admin2',
                    'Lat','Long_','Combined_Key',
                    'Province_State','Country_Region','Population'] 
    d_us = create_df('US_DEATHS', 'Deaths' , {'Deaths':'sum','Population':'sum'}, cols_to_keep)
    
    # Merge
    df_us = pd.merge(c_us, d_us, on=['Date','Province_State'], how='left').reset_index(drop=True)
    df_us.loc[(df_us['Deaths'] != 0),'Fatality Rate'] = round((df_us['Deaths']/df_us['Confirmed']*100),2)
    df_us.loc[(df_us['Confirmed'] != 0),'Confirmed Rate'] = round((df_us['Confirmed']/df_us['Population']*100000),0)
    df_us.loc[(df_us['Deaths'] != 0),'Deaths Rate'] = round((df_us['Deaths']/df_us['Population']*100000),0)
    df_us['Date'] = pd.to_datetime(df_us['Date'], format='%m/%d/%y').dt.strftime('%m/%d/%y')
    
    # Check df
    df_check = df_us[(df_us['Date'] == YESTERDAY.strftime('%m/%d/%y')) & 
                (df_us['Population'] != 0) & 
                (df_us['Confirmed'] != 0)]
    
    if df_check.empty:
        print(f'Data not updated !')
        %stop
    else:
        # Generate
        domain_108(df_us)
        df_save(df_us,'US_MAP','csv')
        
    return df_check

df_us = transform_data(check_data())
df_us.head(5)

Dataframe 108 successfully save in database app-wsr in MongoDB. Time: --- 0.10436606407165527 secnds ---
US_MAP successfully saved in US_MAP
CPU times: user 1.08 s, sys: 185 ms, total: 1.27 s
Wall time: 1.53 s


,Date,Province_State,Confirmed,Deaths,Population,Fatality Rate,Confirmed Rate,Deaths Rate


In [8]:
def create_map(df,geo,kpi,scale,colors):
    file_path = os.path.join(OUTPUT_FOLDER,f'{kpi}.html')
    fig = px.choropleth_mapbox(df, geojson=geo, color=kpi,
                               locations="Province_State", featureidkey="properties.NAME",
                               range_color=scale,
                               color_continuous_scale=colors,
                               mapbox_style="carto-positron", 
                               zoom=3, center={"lat": 37.0902, "lon": -95.7129},
                               labels={'Province_State':'State'}
                              )
    fig.update_layout(margin={"r":0,"l":0,},autosize=True)
    fig.write_html(file_path)
    updateChartCss(file_path, os.path.join(INPUT_FOLDER,'map.css'))

In [9]:
#Confirmed
kpi='Confirmed'
scale = (100,250000)
colors = [
    [0.0, "#FFFFFF"],
    [0.01,"#B3D7D2"],
    [0.1, "#81B1BD"],
    [0.3, "#437F9D"],
    [1.0, "#00354E"]
]
analysis(df_us, kpi)
create_map(df_us,MAP_STATES_20M,kpi,scale,colors)
file_html = f'{kpi}.html'

#Make html visible
token_conf = naas.assets.add(os.path.join(OUTPUT_FOLDER, file_html), params={"inline":True})

/opt/conda/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Confirmed : Min: nan, Max: nan,Average: nan, AvrMax: nan,Med: nan, MedMax: nan
👌 Well done! Your Assets has been sent to production.



Button(description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


In [10]:
#Confirmed Rate
colors = [
    [0.0, "#FFFFFF"],
    [0.01,"#B3D7D2"],
    [0.1, "#81B1BD"],
    [0.3, "#437F9D"],
    [1.0, "#00354E"]
]
scale = (100,1000)
kpi='Confirmed Rate'
analysis(df_us, kpi)
create_map(df_us,MAP_STATES_20M,kpi,scale,colors)
file_html = f'{kpi}.html'

#Make html visible
token_conf_rate = naas.assets.add(os.path.join(OUTPUT_FOLDER, file_html), params={"inline":True})

Confirmed Rate : Min: nan, Max: nan,Average: nan, AvrMax: nan,Med: nan, MedMax: nan


/opt/conda/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning:

Mean of empty slice.

/opt/conda/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars



👌 Well done! Your Assets has been sent to production.



Button(description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


In [11]:
#'Deaths'
colors = [
    [0.0, "#FFFFFF"],
    [0.01,"#B3D7D2"],
    [0.1, "#81B1BD"],
    [0.3, "#437F9D"],
    [1.0, "#00354E"]
]
scale = (0,10000)
kpi='Deaths'
analysis(df_us, kpi)
create_map(df_us,MAP_STATES_20M,kpi,scale,colors)
file_html = f'{kpi}.html'

#Make html visible
token_death = naas.assets.add(os.path.join(OUTPUT_FOLDER, file_html), params={"inline":True})

Deaths : Min: nan, Max: nan,Average: nan, AvrMax: nan,Med: nan, MedMax: nan
👌 Well done! Your Assets has been sent to production.



Button(description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


In [12]:
#'Deaths Rate'
colors = [
    [0.0, "#FFFFFF"],
    [0.01,"#B3D7D2"],
    [0.1, "#81B1BD"],
    [0.3, "#437F9D"],
    [1.0, "#00354E"]
]
scale = (10,100)
kpi='Deaths Rate'
analysis(df_us, kpi)
create_map(df_us,MAP_STATES_20M,kpi,scale,colors)
file_html = f'{kpi}.html'

#Make html visible
token_death_rate = naas.assets.add(os.path.join(OUTPUT_FOLDER, file_html), params={"inline":True})

Deaths Rate : Min: nan, Max: nan,Average: nan, AvrMax: nan,Med: nan, MedMax: nan
👌 Well done! Your Assets has been sent to production.



Button(description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


In [13]:
#Fatality Rate
colors = [
    [0.0, "#FFFFFF"],
    [0.25,"#B3D7D2"],
    [0.5, "#81B1BD"],
    [0.75, "#437F9D"],
    [1.0, "#00354E"]
]
scale = (0,10)
kpi='Fatality Rate'
analysis(df_us, kpi)
create_map(df_us,MAP_STATES_20M,kpi,scale,colors)
file_html = f'{kpi}.html'

#Make html visible
token_fatality_rate = naas.assets.add(os.path.join(OUTPUT_FOLDER, file_html), params={"inline":True})

Fatality Rate : Min: nan, Max: nan,Average: nan, AvrMax: nan,Med: nan, MedMax: nan
👌 Well done! Your Assets has been sent to production.



Button(description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


In [14]:
# html

CONFIRMED = [{'URL':  token_conf,
             'KPI': 'Confirmed',
             'UPPER_FILTER': 'All population'}]

DEATHS = [{'URL':  token_death,
           'KPI': 'Deaths',
           'UPPER_FILTER': 'All population'}]

FATALITY_RATE = [{'URL':  token_fatality_rate,
            'KPI': 'Fatality Rate',
            'UPPER_FILTER': 'All population'}]

CONFIRMED_RATE = [{'URL':  token_conf_rate,
            'KPI': 'Confirmed Ratio',
            'UPPER_FILTER': '100k inhab'}]

DEATH_RATE = [{'URL':  token_death_rate,
             'KPI': 'Deaths Ratio',
             'UPPER_FILTER': '100k inhab'}]

#Concat list to create Series
LIST_KPIS = (CONFIRMED + DEATHS + FATALITY_RATE + CONFIRMED_RATE + DEATH_RATE)

#Create Dataframe
DOMAIN = pd.DataFrame(LIST_KPIS)
DOMAIN.loc[:, 'VALUE'] = ("<iframe src='" +
                          DOMAIN['URL'] +
                          ("' frameborder='0' style='overflow:hidden;"
                           "height:85vh;width:100%' height='100%' "
                           "width='100%'></iframe> <style>.modebar"
                           "{display: none!important;}</style>"))
if USE_MONGO:
    naas_drivers.mongo.send(DOMAIN,'109',DB_APP,True)
DOMAIN

Dataframe 109 successfully save in database app-wsr in MongoDB. Time: --- 0.019133567810058594 secnds ---


,URL,KPI,UPPER_FILTER,VALUE
0,https://public.naas.ai/Ym9iLTQwY2FzaHN0b3J5LTJ...,Confirmed,All population,<iframe src='https://public.naas.ai/Ym9iLTQwY2...
1,https://public.naas.ai/Ym9iLTQwY2FzaHN0b3J5LTJ...,Deaths,All population,<iframe src='https://public.naas.ai/Ym9iLTQwY2...
2,https://public.naas.ai/Ym9iLTQwY2FzaHN0b3J5LTJ...,Fatality Rate,All population,<iframe src='https://public.naas.ai/Ym9iLTQwY2...
3,https://public.naas.ai/Ym9iLTQwY2FzaHN0b3J5LTJ...,Confirmed Ratio,100k inhab,<iframe src='https://public.naas.ai/Ym9iLTQwY2...
4,https://public.naas.ai/Ym9iLTQwY2FzaHN0b3J5LTJ...,Deaths Ratio,100k inhab,<iframe src='https://public.naas.ai/Ym9iLTQwY2...
